In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, Activation
from keras import optimizers
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
from glob import glob
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc
from math import pi
import random
from keras.layers.advanced_activations import PReLU,LeakyReLU
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from tensorflow.keras.optimizers import SGD
#from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Concatenate
from numpy import zeros
from numpy import ones
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
import csv
from keras.initializers import Constant
from keras import backend as K
import time, datetime
!pip install -U tensorflow-addons
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_addons as tfa

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 69, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/__init__.py", line 91, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames

ModuleNotFoundError: ignored

In [ ]:
GAN_Model_Load_flag = 0 #=1(load Model Weight), =0(Not Load)
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir(os.path.join("/content/gdrive/My Drive/Colab Notebooks/GAN/1100122-IZGAN_V5_7_256"))
dirpath = os.getcwd()
load_folder_name = 'GAN_Sim_Data' + '0'
save_folder_name = 'GAN_Sim_Data' + '1'
os.mkdir(save_folder_name)
dirpath_load = (dirpath + '/' + load_folder_name)
dirpath_save = (dirpath + '/' + save_folder_name)
load_model_name = (dirpath_load + '/IZGAN_V5.h5')
save_model_name = (dirpath_save + '/IZGAN_V5.h5')

In [ ]:
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
train_GAN_epochs = 700
train_GAN_batch_size = 1
train_GAN_pic = 400
train_GAN_pict = 40
train_GAN_discriminator_epochs = 5 
train_GAN_generator_epochs = 1
test_model_epochs = 1000
test_model_pict = 1 
image_size = 256
gridx = int(image_size/4) 
gridy = int(image_size/8) 
image_shape = (image_size, image_size, 1)
optimizer_build_generator = "rmsprop" 
optimizer_build_discriminator = SGD(0.00002) 
optimizer_build_GAN = Adam(0.0002) 
build_generator_loss = root_mean_squared_error 
build_discriminator_loss = 'mse' 
build_GAN_gen_loss = root_mean_squared_error 
build_GAN_dis_loss = 'mse'
rle_d_patience = 25 
rle_g_patience = 50
labelsize_x_discriminator = 8
labelsize_y_discriminator = 8
monitortype = "val_loss" 

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [ ]:
def load_data(pic):
  a = np.linspace(-1*(image_size/2), (image_size/2)-1, image_size)
  xv,yv = np.meshgrid( a , a )
  yv = yv*-1
  rho,theta = cart2pol( xv , yv )
  m =rho.max()*0.7
  rho = rho/m
  rho[rho>1]=np.nan

  X = np.zeros((pic, image_size, image_size))
  Y = np.zeros((pic, image_size, image_size))
  c_train = np.zeros((pic,32))
  for i in range(0,pic,1):
    c =(np.random.rand(1,32)*2-1)/2
    c = c.round(6)
    c_train[i,:] = c[0,:] 
    phi =c[0,0]*1 \
        +c[0,1]* 2 * rho * np.sin(theta) \
        +c[0,2]* 2 * rho * np.cos(theta)\
        +c[0,3]*np.sqrt(6)*rho**2 * np.sin(2*theta)\
        +c[0,4]*np.sqrt(3)*(2*rho**2 - 1)\
        +c[0,5]*np.sqrt(6)*rho**2 * np.cos(2*theta)\
        +c[0,6]*np.sqrt(8)*rho**3 * np.sin(3*theta)\
        +c[0,7]*np.sqrt(8)*(3*rho**3 - 2*rho) * np.sin(theta)\
        +c[0,8]*np.sqrt(8)*(3*rho**3 - 2*rho) * np.cos(theta)\
        +c[0,9]*np.sqrt(8)*rho**3 * np.cos(3*theta)\
        +c[0,10]*np.sqrt(10)*rho**4* np.sin(4*theta)\
        +c[0,11]*np.sqrt(10)*(4*rho**4 - 3*rho**2) * np.sin(2*theta)\
        +c[0,12]*np.sqrt(5)*(6*rho**4 - 6*rho**2 + 1)\
        +c[0,13]*np.sqrt(10)*(4*rho**4 - 3*rho**2) * np.cos(2*theta)\
        +c[0,14]*np.sqrt(10)*rho**4* np.cos(4*theta)\
        +c[0,15]*np.sqrt(12)*rho**5* np.sin(5*theta)\
        +c[0,16]*np.sqrt(12)*(5*rho**5 - 4*rho**3) * np.sin(3*theta)\
        +c[0,17]*np.sqrt(12)*(10*rho**5 - 12*rho**3 + 3*rho) * np.sin(theta)\
        +c[0,18]*np.sqrt(12)*(10*rho**5 - 12*rho**3 + 3*rho) * np.cos(theta)\
        +c[0,19]*np.sqrt(12)*(5*rho**5 - 4*rho**3) * np.cos(3*theta)\
        +c[0,20]*np.sqrt(12)*rho**5* np.cos(5*theta)\
        +c[0,21]*np.sqrt(14)*rho**6* np.sin(6*theta)\
        +c[0,22]*np.sqrt(14)*(6*rho**6 - 5*rho**4) * np.sin(4*theta)\
        +c[0,23]*np.sqrt(14)*(15*rho**6 - 20*rho**4 + 6*rho**2) * np.sin(2*theta)\
        +c[0,24]*np.sqrt(7)*(20*rho**6 - 30*rho**4 + 12*rho**2 - 1)\
        +c[0,25]*np.sqrt(14)*(15*rho**6 - 20*rho**4 + 6*rho**2) * np.cos(2*theta)\
        +c[0,26]*np.sqrt(14)*(6*rho**6 - 5*rho**4) * np.cos(4*theta)\
        +c[0,27]*np.sqrt(14)*rho**6* np.cos(6*theta)\
        +c[0,28]*4 * rho**7* np.sin(7*theta)\
        +c[0,29]*4 *(7*rho**7 - 6*rho**5) * np.sin(5*theta)\
        +c[0,30]*4 *(21*rho**7 - 30*rho**5 + 10*rho**3) * np.sin(3*theta)\
        +c[0,31]*4 *(35*rho**7 - 60*rho**5 + 30*rho**3 - 4*rho) * np.sin(theta)
    fring = 4*np.cos(phi*2*pi/2)**2
    fring2 = 4*np.cos((phi*2*pi/2)+(pi/4))**2
    for mm in range(image_size):
      for nn in range(image_size):
        if fring2[mm,nn] == 0:
          fring2[mm,nn] = 0.00000001
    fring3 = fring / fring2
    fring3 = np.log(fring3)
    fring3[np.isnan(fring3)] = 0
    X[i] = fring3

    samenum = 0
    z_pos = 0
    flag = 0
    z = np.zeros((image_size, image_size))
    for r in range(0,8,1):
        for j in range(0+(gridy*r),gridy+(gridy*r),1):
            for k in range(0,image_size,1):
                z[j][k]= c[0,z_pos]
                flag = flag + 1
                samenum = samenum + 1
                if samenum >= gridx:
                    samenum = 0
                    z_pos = z_pos + 1
                    if z_pos >= 4 + (4*r):
                       z_pos = 0 + (4*r)
        
    Y[i] = z
  x_train = X.reshape(pic, image_size,image_size,1)
  y_train = Y.reshape(pic, image_size,image_size,1)
  del X
  del Y
  return (x_train, y_train, c_train)
x_train, y_train, c_train = load_data(10)

In [ ]:
def test_data(pict):
  a = np.linspace(-1*(image_size/2), (image_size/2)-1, image_size)
  xv,yv = np.meshgrid( a , a )
  yv = yv*-1
  rho,theta = cart2pol( xv , yv )
  m =rho.max()*0.7
  rho = rho/m
  rho[rho>1]=np.nan

  X = np.zeros((pict, image_size, image_size))
  Y = np.zeros((pict, image_size, image_size))
  c_test = np.zeros((pict,32))

  for i in range(0,pict,1):
      c =(np.random.rand(1,32)*2-1)/2
      c = c.round(6)
      c_test[i,:] = c[0,:]
   
      phi =c[0,0]*1 \
          +c[0,1]* 2 * rho * np.sin(theta) \
          +c[0,2]* 2 * rho * np.cos(theta)\
          +c[0,3]*np.sqrt(6)*rho**2 * np.sin(2*theta)\
          +c[0,4]*np.sqrt(3)*(2*rho**2 - 1)\
          +c[0,5]*np.sqrt(6)*rho**2 * np.cos(2*theta)\
          +c[0,6]*np.sqrt(8)*rho**3 * np.sin(3*theta)\
          +c[0,7]*np.sqrt(8)*(3*rho**3 - 2*rho) * np.sin(theta)\
          +c[0,8]*np.sqrt(8)*(3*rho**3 - 2*rho) * np.cos(theta)\
          +c[0,9]*np.sqrt(8)*rho**3 * np.cos(3*theta)\
          +c[0,10]*np.sqrt(10)*rho**4* np.sin(4*theta)\
          +c[0,11]*np.sqrt(10)*(4*rho**4 - 3*rho**2) * np.sin(2*theta)\
          +c[0,12]*np.sqrt(5)*(6*rho**4 - 6*rho**2 + 1)\
          +c[0,13]*np.sqrt(10)*(4*rho**4 - 3*rho**2) * np.cos(2*theta)\
          +c[0,14]*np.sqrt(10)*rho**4* np.cos(4*theta)\
          +c[0,15]*np.sqrt(12)*rho**5* np.sin(5*theta)\
          +c[0,16]*np.sqrt(12)*(5*rho**5 - 4*rho**3) * np.sin(3*theta)\
          +c[0,17]*np.sqrt(12)*(10*rho**5 - 12*rho**3 + 3*rho) * np.sin(theta)\
          +c[0,18]*np.sqrt(12)*(10*rho**5 - 12*rho**3 + 3*rho) * np.cos(theta)\
          +c[0,19]*np.sqrt(12)*(5*rho**5 - 4*rho**3) * np.cos(3*theta)\
          +c[0,20]*np.sqrt(12)*rho**5* np.cos(5*theta)\
          +c[0,21]*np.sqrt(14)*rho**6* np.sin(6*theta)\
          +c[0,22]*np.sqrt(14)*(6*rho**6 - 5*rho**4) * np.sin(4*theta)\
          +c[0,23]*np.sqrt(14)*(15*rho**6 - 20*rho**4 + 6*rho**2) * np.sin(2*theta)\
          +c[0,24]*np.sqrt(7)*(20*rho**6 - 30*rho**4 + 12*rho**2 - 1)\
          +c[0,25]*np.sqrt(14)*(15*rho**6 - 20*rho**4 + 6*rho**2) * np.cos(2*theta)\
          +c[0,26]*np.sqrt(14)*(6*rho**6 - 5*rho**4) * np.cos(4*theta)\
          +c[0,27]*np.sqrt(14)*rho**6* np.cos(6*theta)\
          +c[0,28]*4 * rho**7* np.sin(7*theta)\
          +c[0,29]*4 *(7*rho**7 - 6*rho**5) * np.sin(5*theta)\
          +c[0,30]*4 *(21*rho**7 - 30*rho**5 + 10*rho**3) * np.sin(3*theta)\
          +c[0,31]*4 *(35*rho**7 - 60*rho**5 + 30*rho**3 - 4*rho) * np.sin(theta)
 
    
      fring = 4*np.cos(phi*2*pi/2)**2
      fring2 = 4*np.cos((phi*2*pi/2)+(pi/4))**2
      for mm in range(image_size):
        for nn in range(image_size):
          if fring2[mm,nn] == 0:
            fring2[mm,nn] = 0.00000001
      fring3 = fring / fring2
      fring3 = np.log(fring3)
      fring3[np.isnan(fring3)] = 0
      X[i] = fring3

      samenum = 0
      z_pos = 0
      flag = 0
      z = np.zeros((image_size, image_size))
      for r in range(0,8,1):
          for j in range(0+(gridy*r),gridy+(gridy*r),1):
              for k in range(0,image_size,1):
                  z[j][k]= c[0,z_pos]
                  flag = flag + 1
                  samenum = samenum + 1
                  if samenum >= gridx:
                      samenum = 0
                      z_pos = z_pos + 1
                      if z_pos >= 4 + (4*r):
                        z_pos = 0 + (4*r)
      Y[i] = z
  x_test = X.reshape(pict, image_size,image_size,1)
  y_test = Y.reshape(pict, image_size,image_size,1)
  del X
  del Y
  return (x_test, y_test, c_test)

In [ ]:
def build_generator(image_shape=(image_shape)):

    inputs = Input(image_shape)
    s = inputs

    c1 = Conv2D(32, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (s)
    c1 = tfa.layers.InstanceNormalization(axis=3)(c1)
    c1 = Conv2D(32, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c1)
    c1 = tfa.layers.InstanceNormalization(axis=3)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)


    c2 = Conv2D(64, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p1)
    c2 = tfa.layers.InstanceNormalization(axis=3)(c2)
    c2 = Conv2D(64, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c2)
    c2 = tfa.layers.InstanceNormalization(axis=3)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(128, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p2)
    c3 = tfa.layers.InstanceNormalization(axis=3)(c3)
    c3 = Conv2D(128, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c3)
    c3 = tfa.layers.InstanceNormalization(axis=3)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(256, (2, 2),strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p3)
    c4 = tfa.layers.InstanceNormalization(axis=3)(c4)
    c4 = Conv2D(256, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c4)
    c4 = tfa.layers.InstanceNormalization(axis=3)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(512, (2, 2),strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p4)
    c5 = tfa.layers.InstanceNormalization(axis=3)(c5)
    c5 = Conv2D(512, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c5)
    c5 = tfa.layers.InstanceNormalization(axis=3)(c5)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

    c6 = Conv2D(1024, (2, 2),strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p5)
    c6 = tfa.layers.InstanceNormalization(axis=3)(c6)
    c6 = Conv2D(1024, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c6)
    c6 = tfa.layers.InstanceNormalization(axis=3)(c6)
    p6 = MaxPooling2D(pool_size=(2, 2)) (c6)

    c7 = Conv2D(2048, (2, 2),strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (p6)
    c7 = tfa.layers.InstanceNormalization(axis=3)(c7)
    c7 = Conv2D(2048, (2, 2), strides=(1, 1), activation='tanh', kernel_initializer='he_normal', padding='same') (c7)
    c7 = tfa.layers.InstanceNormalization(axis=3)(c7)
    p7 = MaxPooling2D(pool_size=(2, 2)) (c7)

    c8 = Conv2D(4096, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (p7)
    c8 = PReLU(alpha_initializer=Constant(value=0.2))(c8)
    c8 = Conv2D(4096, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c8)
    c8 = PReLU(alpha_initializer=Constant(value=0.2))(c8)
    
    u9 = Conv2DTranspose(2048, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c7])
    c9 = Conv2D(2048, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u9)
    c9 = PReLU(alpha_initializer=Constant(value=0.2))(c9)
    c9 = tfa.layers.InstanceNormalization(axis=3)(c9)
    c9 = Conv2D(2048, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c9)
    c9 = PReLU(alpha_initializer=Constant(value=0.2))(c9)
    c9 = tfa.layers.InstanceNormalization(axis=3)(c9)

    u10 = Conv2DTranspose(1024, (2, 2), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c6])
    c10 = Conv2D(1024, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u10)
    c10 = PReLU(alpha_initializer=Constant(value=0.2))(c10)
    c10 = tfa.layers.InstanceNormalization(axis=3)(c10)
    c10 = Conv2D(1024, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c10)
    c10 = PReLU(alpha_initializer=Constant(value=0.2))(c10)
    c10 = tfa.layers.InstanceNormalization(axis=3)(c10)

    u11 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c5])
    c11 = Conv2D(512, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u11)
    c11 = PReLU(alpha_initializer=Constant(value=0.2))(c11)
    c11 = tfa.layers.InstanceNormalization(axis=3)(c11)
    c11 = Conv2D(512, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c11)
    c11 = PReLU(alpha_initializer=Constant(value=0.2))(c11)
    c11 = tfa.layers.InstanceNormalization(axis=3)(c11)

    u12 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c11)
    u12 = concatenate([u12, c4])
    c12 = Conv2D(256, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u12)
    c12 = PReLU(alpha_initializer=Constant(value=0.2))(c12)
    c12 = tfa.layers.InstanceNormalization(axis=3)(c12)
    c12 = Conv2D(256, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c12)
    c12 = PReLU(alpha_initializer=Constant(value=0.2))(c12)
    c12 = tfa.layers.InstanceNormalization(axis=3)(c12)

    u13 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c12)
    u13 = concatenate([u13, c3])
    c13 = Conv2D(128, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u13)
    c13 = PReLU(alpha_initializer=Constant(value=0.2))(c13)
    c13 = tfa.layers.InstanceNormalization(axis=3)(c13)
    c13 = Conv2D(128, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (c13)
    c13 = PReLU(alpha_initializer=Constant(value=0.2))(c13)
    c13 = tfa.layers.InstanceNormalization(axis=3)(c13)

    u14 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c13)
    u14 = concatenate([u14, c2])
    c14 = Conv2D(64, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u14)
    c14 = PReLU(alpha_initializer=Constant(value=0.2))(c14)
    c14 = tfa.layers.InstanceNormalization(axis=3)(c14)
    c14 = Conv2D(64, (2, 2),strides=(1, 1), kernel_initializer='he_normal', padding='same') (c14)
    c14 = PReLU(alpha_initializer=Constant(value=0.2))(c14)
    c14 = tfa.layers.InstanceNormalization(axis=3)(c14)

    u15 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c14)
    u15 = concatenate([u15, c1], axis=3)
    c15 = Conv2D(32, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (u15)
    c15 = PReLU(alpha_initializer=Constant(value=0.2))(c15)
    c15 = tfa.layers.InstanceNormalization(axis=3)(c15)
    c15 = Conv2D(32, (2, 2), strides=(1, 1), kernel_initializer='he_normal', padding='same') (c15)
    c15 = PReLU(alpha_initializer=Constant(value=0.2))(c15)
    c15 = tfa.layers.InstanceNormalization(axis=3)(c15)

    c16 = Conv2D(1, (1, 1)) (c15)
    outputs = PReLU(alpha_initializer=Constant(value=0.2))(c16)

    model = Model(inputs=[inputs], outputs=[outputs], name='Generator')
    model.compile(loss=build_generator_loss, optimizer=optimizer_build_generator,metrics=['accuracy'])
    return model

generator = build_generator()
generator.summary()

In [ ]:
def build_discriminator(image_shape=(image_shape)):
    
    in_src_image = Input(shape=image_shape)
    in_target_image = Input(shape=image_shape)
    merged = Concatenate()([in_src_image, in_target_image])
    
    d = Conv2D(32, (4,4), strides=(2,2),  padding='same')(merged)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(64, (4,4), strides=(2,2),  padding='same')(d)
    d = BatchNormalization(momentum=0.8)(d)
    d = LeakyReLU(alpha=0.2)(d)
	
    d = Conv2D(128, (4,4), strides=(2,2),  padding='same')(d)
    d = BatchNormalization(momentum=0.8)(d)
    d = LeakyReLU(alpha=0.2)(d)
	
    d = Conv2D(256, (4,4), strides=(2,2), padding='same')(d)
    d = tfa.layers.InstanceNormalization(axis=3)(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(512, (4,4), strides=(2,2),padding='same')(d)
    d = BatchNormalization(momentum=0.8)(d)
    d = LeakyReLU(alpha=0.2)(d)
	
    d = Conv2D(512, (4,4), padding='same')(d)
    d = BatchNormalization(momentum=0.8)(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(1, (4,4), padding='same')(d)
    patch_out = Activation('sigmoid')(d)

    model = Model([in_src_image, in_target_image], patch_out, name='Discriminator')
    model.compile(loss=build_discriminator_loss, optimizer=optimizer_build_discriminator,metrics=['accuracy'])
    return model
   
image_shape =(image_size,image_size,1)
discriminator = build_discriminator(image_shape)
discriminator.summary()

In [ ]:
def build_GAN(discriminator, generator):
    discriminator.trainable=False
    GAN_input = Input(shape=(image_shape))
    x = generator(GAN_input)
    GAN_output= discriminator([x,GAN_input])
    GAN = Model(inputs=GAN_input, outputs=[GAN_output,x], name='PhaseGAN')
    if GAN_Model_Load_flag == 1 :
      GAN.load_weights(load_model_name, by_name = True)
      print('GAN model has loaded')
    GAN.compile(loss=[build_GAN_dis_loss,build_GAN_gen_loss], loss_weights=[1, 100], optimizer=optimizer_build_GAN,metrics=['accuracy'])
    return GAN
 
GAN = build_GAN(discriminator, generator)
GAN.summary()

In [ ]:
def draw_images(generator, epoch, examples=25, dim=(5,5), figsize=(10,10)):

    x_train, y_train, c_train = load_data(pic=1)
    generated_images = generator.predict(x_train)
    generated_images = generated_images.reshape(image_size,image_size)
    x_train = x_train.reshape(image_size,image_size)
    y_train = y_train.reshape(image_size,image_size)

    print('x_train Data最大值 : %f' % x_train.max())
    print('x_train Data最小值 : %f' % x_train.min())
    print('y_train Data最大值 : %f' % y_train.max())
    print('y_train Data最小值 : %f' % y_train.min())
    print('generated_images 預測最大值 : %f' % generated_images.max())
    print('generated_images 預測最小值 : %f' % generated_images.min())
    
    c_csv = pd.DataFrame(c_train[0,:],columns=['c_train'])
    c_csv.to_csv(dirpath_save + "/c_train_%d.csv" % epoch)
    """  
    with open(dirpath_save + '/x_train_%d.csv' % epoch, 'w', newline='') as fx:
      writer = csv.writer(fx)
      writer.writerows(x_train)
    fx.close()

    with open(dirpath_save + '/y_train_%d.csv' % epoch, 'w', newline='') as fy:
      writer = csv.writer(fy)
      writer.writerows(y_train)
    fy.close()
    """
    with open(dirpath_save + '/generated_images_%d.csv' % epoch, 'w', newline='') as fg:
      writer = csv.writer(fg)
      writer.writerows(generated_images)
    fg.close()
    """
    print('Save X_Train Images ', epoch)
    imshow(x_train)
    plt.savefig(dirpath_save + "/x_train_%d.png" % epoch)
    plt.show()

    print('Save Y_Train Images ', epoch)
    imshow(y_train)
    plt.savefig(dirpath_save + "/y_train_%d.png" % epoch)
    plt.show()
    """
    print('Save Generated Images ', epoch)
    imshow(generated_images)
    plt.savefig(dirpath_save + "/generated_images_%d.png" % epoch)
    plt.show()
    #--------------------------------------------------------------------------------------------
    x_test, y_test, c_test = test_data(pict=1)
    generated_images_t = generator.predict(x_test)
    generated_images_t = generated_images_t.reshape(image_size,image_size)
    x_test = x_test.reshape(image_size,image_size)
    y_test = y_test.reshape(image_size,image_size)

    print('x_test Data最大值 : %f' % x_test.max())
    print('x_test Data最小值 : %f' % x_test.min())
    print('y_test Data最大值 : %f' % y_test.max())
    print('y_test Data最小值 : %f' % y_test.min())
    print('generated_images預測最大值 : %f' % generated_images_t.max())
    print('generated_images預測最小值 : %f' % generated_images_t.min())
    
    c_csv = pd.DataFrame(c_test[0,:],columns=['c_test'])
    c_csv.to_csv(dirpath_save + "/c_test_%d.csv" % epoch)
    """
    with open(dirpath_save + '/x_test_%d.csv' % epoch, 'w', newline='') as fx:
      writer = csv.writer(fx)
      writer.writerows(x_test)
    fx.close()

    with open(dirpath_save + '/y_test_%d.csv' % epoch, 'w', newline='') as fy:
      writer = csv.writer(fy)
      writer.writerows(y_test)
    fy.close()
    """
    with open(dirpath_save + '/generated_images_t_%d.csv' % epoch, 'w', newline='') as fg:
      writer = csv.writer(fg)
      writer.writerows(generated_images_t)
    fg.close()
    """
    print('Save X_Test Images ', epoch)
    imshow(x_test)
    plt.savefig(dirpath_save + "/x_test_%d.png" % epoch)
    plt.show()

    print('Save Y_Test Images ', epoch)
    imshow(y_test)
    plt.savefig(dirpath_save + "/y_test_%d.png" % epoch)
    plt.show()
    """
    print('Save Generated Images Test ', epoch)
    imshow(generated_images_t)
    plt.savefig(dirpath_save + "/generated_images_t_%d.png" % epoch)
    plt.show()
    #==============================================================================================

    df_gen = pd.read_csv(dirpath_save + '/generated_images_%d.csv' % epoch, header = None)
    df_gen = df_gen.values 

    gen_c = np.zeros(32) 
    gen_c = gen_c.reshape([1,32])

    cut_raw = 1
    cut_col = 1

    c_pos = 0
    for j in range(0,8,1):
      for i in range(0,4,1):
        df_tmp = df_gen[gridy*j+cut_raw:gridy*(j+1)-cut_raw, gridx*i+cut_col:gridx*(i+1)-cut_col]
        #print(df_tmp)
        gen_c[0,c_pos] = df_tmp.sum()/((gridy-2*cut_raw)*(gridx-2*cut_col))
        c_pos = c_pos + 1
    gen_c[0,0] = 0

    df_c = pd.read_csv(dirpath_save + '/c_train_%d.csv' % epoch, header = 0)
    df_c = df_c.values 
    df_c = df_c[:,1]
    df_c[0] = 0

    mse=np.sum((df_c-gen_c)**2)/32
    print('mse = ', mse) 
    rmse=mse ** 0.5 
    print('rmse = ', rmse)

    #==============================================================================================
    df_gen = pd.read_csv(dirpath_save + '/generated_images_t_%d.csv' % epoch, header = None)
    df_gen = df_gen.values 

    gen_c = np.zeros(32) 
    gen_c = gen_c.reshape([1,32])

    cut_raw = 1
    cut_col = 1

    c_pos = 0
    for j in range(0,8,1):
      for i in range(0,4,1):
        df_tmp = df_gen[gridy*j+cut_raw:gridy*(j+1)-cut_raw, gridx*i+cut_col:gridx*(i+1)-cut_col]
        gen_c[0,c_pos] = df_tmp.sum()/((gridy-2*cut_raw)*(gridx-2*cut_col))
        c_pos = c_pos + 1
    gen_c[0,0] = 0

    df_c = pd.read_csv(dirpath_save + '/c_test_%d.csv' % epoch, header = 0)
    df_c = df_c.values 
    df_c = df_c[:,1]
    df_c[0] = 0

    mse_t=np.sum((df_c-gen_c)**2)/32
    print('mse_t = ', mse_t) 
    rmse_t=mse_t ** 0.5 
    print('rmse_t = ', rmse_t)

    return (mse, rmse, mse_t, rmse_t)

In [ ]:
es = EarlyStopping(monitor=monitortype, mode="min", verbose=1, patience=20)
rle_d = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=rle_d_patience,verbose=1,min_lr=1e-15)
rle_g = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=rle_g_patience,verbose=1,min_lr=1e-10)
mc = ModelCheckpoint(save_model_name, monitor=monitortype, mode="min", verbose=2, save_best_only=True, save_weights_only=True)

In [ ]:
def train_GAN(epochs, batch_size, pic, pict):

  mse_avg = np.zeros(epochs)
  rmse_avg = np.zeros(epochs)
  mse_t_avg = np.zeros(epochs)
  rmse_t_avg = np.zeros(epochs)

  pic_tmp = pic
  pict_tmp = pict
  
  generator= build_generator()
  discriminator= build_discriminator()
  GAN = build_GAN(discriminator, generator)
  history_all = {}
  
  for i in tqdm(range(1, epochs+1)):
        print("Round %d" %i)     
        X_train, y_train, c_train = load_data(pic)
        X_test, y_test, c_test = test_data(pict)
        
        fake_images = generator.predict(X_train)
        real_images = y_train

        fake_images_t = generator.predict(X_test)
        real_images_t = y_test
 
        label_fake = zeros((pic,labelsize_x_discriminator,labelsize_y_discriminator))
        label_fake = label_fake.reshape((pic,labelsize_x_discriminator,labelsize_y_discriminator,1))
        label_real = ones((pic,labelsize_x_discriminator,labelsize_y_discriminator))
        label_real = label_real.reshape((pic,labelsize_x_discriminator,labelsize_y_discriminator,1))

        label_fake_t = zeros((pict,labelsize_x_discriminator,labelsize_y_discriminator))
        label_fake_t = label_fake_t.reshape((pict,labelsize_x_discriminator,labelsize_y_discriminator,1))
        label_real_t = ones((pict,labelsize_x_discriminator,labelsize_y_discriminator))
        label_real_t = label_real_t.reshape((pict,labelsize_x_discriminator,labelsize_y_discriminator,1)) 

        inp = np.concatenate([X_train, X_train])
        X = np.concatenate([fake_images, real_images])
        y = np.concatenate([label_fake, label_real])

        inp_t = np.concatenate([X_test, X_test])
        X_t = np.concatenate([fake_images_t, real_images_t])
        y_t = np.concatenate([label_fake_t, label_real_t])
        for j in range(1, batch_size+1):

          discriminator.trainable=True
          history = discriminator.fit([inp,X], y, epochs=train_GAN_discriminator_epochs, batch_size=32, verbose=1, validation_data=([inp_t,X_t], y_t))
          discriminator.trainable=False
          history = GAN.fit(X_train, [label_real,y_train], epochs=train_GAN_generator_epochs, batch_size=4, callbacks=[rle_g,es,mc], validation_data=(X_test, [label_real_t,y_test]))
        
        if len(history_all) == 0:
            history_all = {key: [] for key in history.history}
        for key in history_all:
          history_all[key].extend(history.history[key])
  
        if i == 1 or i % 10 == 0:
          mse, rmse, mse_t, rmse_t = draw_images(generator, i)
          mse_avg[i-1] = mse
          rmse_avg[i-1] = rmse
          print("mse_avg =", np.sum(mse_avg)/((i/10)+1))
          print("rmse_avg =", np.sum(rmse_avg)/((i/10)+1))


In [ ]:
def test_model(epochs, pict, cut_raw, cut_col):
  
  mse_avg = np.zeros(epochs)
  rmse_avg = np.zeros(epochs)

  generator= build_generator()
  discriminator= build_discriminator()
  GAN = build_GAN(discriminator, generator)
  
  history_all = {}

  for i in tqdm(range(1, epochs+1)):
    x_train, y_train, c_train = load_data(pict)
    generated_images = generator.predict(x_train)
    generated_images = generated_images.reshape(image_size,image_size)
    x_train = x_train.reshape(image_size,image_size)
    y_train = y_train.reshape(image_size,image_size)
    
    gen_c = np.zeros(32) 
    gen_c = gen_c.reshape([1,32])
    
    df_gen = generated_images

    c_pos = 0
    for j in range(0,8,1):
      for k in range(0,4,1):
        df_tmp = df_gen[gridy*j+cut_raw:gridy*(j+1)-cut_raw, gridx*k+cut_col:gridx*(k+1)-cut_col]
        gen_c[0,c_pos] = df_tmp.sum()/((gridy-2*cut_raw)*(gridx-2*cut_col))
        c_pos = c_pos + 1
    gen_c[0,0] = 0

    df_c = c_train
    df_c[0,0] = 0

    mse=np.sum((df_c-gen_c)**2)/32
    mse_avg[i-1] = mse
    rmse=mse ** 0.5 
    rmse_avg[i-1] = rmse

  print("rmse_avg_%d =" % i, np.sum(rmse_avg)/i)


In [ ]:
train_GAN(train_GAN_epochs,train_GAN_batch_size, train_GAN_pic, train_GAN_pict)